In [ ]:
from base64 import b64decode
import requests
import json
import pandas as pd

# Hide the authKey in .env file
import os
from dotenv import load_dotenv
load_dotenv()  # Load the variables from .env
authKey = os.getenv('authKey')

In [ ]:
url_df = pd.DataFrame()
url_df['Job Links'] = pd.read_json('url_list.json')

In [ ]:
# Attempting to focus on only the data science job posts
# Drop duplicate urls.
url_df.drop_duplicates(inplace=True, ignore_index=True)

# Drop urls with software in the url. 
drop_software = url_df[url_df['Job Links'].str.contains("software", na=False)].index
url_df.drop(index=drop_software, axis=0, inplace=True)

# Drop urls with engineer in the url. 
engineer_df = url_df[url_df['Job Links'].str.contains("engineer", na=False)]
url_df.drop(index=engineer_df.index, axis=0, inplace=True)

In [ ]:
url_df.to_csv('url_df.csv', index=False)

In [ ]:
# If it doesn't already exist, then un-comment-out the below 
#raw_df = pd.DataFrame()

In [ ]:
# Grab the data for each of the urls
# Fetch data then extract data
# This uses a Rest API 
for url in url_df['Job Links']:

    posting_r = requests.post(
        "https://api.zyte.com/v1/extract",
        auth=(authKey, ""),
        json={
            "url": url,
            "httpResponseBody": True,
            "jobPosting": True,
            "jobPostingOptions": {"extractFrom":"httpResponseBody"},
        },
    )
    http_posting_body: bytes = b64decode(
        posting_r.json()["httpResponseBody"])
    jobPosting = posting_r.json()["jobPosting"]

    # Format to df and add row
    add_job = pd.json_normalize(jobPosting)
    raw_df = raw_df.merge(add_job, how='outer')

In [ ]:
raw_df.to_csv('raw_info_df.csv', index = False)